# TESTING ENVS

In [1]:
from myosuite.utils import gym
import skvideo.io
import numpy as np
import os
from stable_baselines3 import PPO
from tqdm.notebook import tqdm

In [ ]:
from IPython.display import HTML
from base64 import b64encode
 
def show_video(video_path, video_width = 400):
   
  video_file = open(video_path, "r+b").read()
 
  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"""<video autoplay width={video_width} controls><source src="{video_url}"></video>""")
 

In [2]:
env = gym.make('myoHandWheelHoldFixed-v0')
env.reset()

    MyoSuite: A contact-rich simulation suite for musculoskeletal motor control
        Vittorio Caggiano, Huawei Wang, Guillaume Durandau, Massimo Sartori, Vikash Kumar
        L4DC-2019 | https://sites.google.com/view/myosuite
    


(array([ 0.    , -0.0111, -0.0134,  0.0102, -0.75  ,  0.1511, -0.    ,
         0.    , -0.152 ,  0.6447,  0.    , -0.552 , -0.3131,  0.    ,
        -0.3817,  0.2514, -0.37  ,  2.094 ,  1.1118,  0.1257,  0.1828,
        -0.1414,  0.6968, -0.5062,  0.3569,  0.    ,  0.7462,  0.0262,
         1.139 ,  1.3432,  0.7462, -0.0052,  1.0604,  1.2961,  0.8169,
         0.0131,  0.6598,  1.571 ,  0.5498, -0.0654,  0.5184,  1.2411,
         0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
         0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
         0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
         0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
         0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
         0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
         0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
         0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
      

In [ ]:
model = PPO("MlpPolicy", env, verbose=0)

In [ ]:
model.learn(total_timesteps=1e4)
# to train to convergence use more iterations e.g.
# model.learn(total_timesteps=1e7)

In [ ]:
model.save("WheelDist_policy")

In [ ]:
# env.sim.renderer.set_viewer_settings(
#            render_actuator=True,
#            render_tendon=True
#        )

pi = PPO.load("WheelDist_policy")

env = gym.make('myoHandWheelHoldFixed-v0')
env.reset()

In [ ]:
# render
env.reset()
frames = []
for _ in range(300):
    frames.append(env.sim.renderer.render_offscreen(width=400, height=400, camera_id=3))
    o = env.get_obs()
    a = pi.predict(o)[0]
    next_o, r, done, *_, ifo = env.step(
        a
    )  # take an action based on the current observation

import skvideo.io
import os
os.makedirs("videos", exist_ok=True)
# make a local copy
skvideo.io.vwrite(
    "videos/RockPose.mp4",
    np.asarray(frames),
    outputdict={"-pix_fmt": "yuv420p", "-r": "10"},
)
show_video('videos/RockPose.mp4')

In [ ]:
# evaluate policy
all_rewards = []
for _ in tqdm(range(5)): # 5 random targets
  ep_rewards = []
  done = False
  obs = env.reset()
  done = False
  for _ in range(100):
      obs = env.obsdict2obsvec(env.obs_dict, env.obs_keys)[1]
      # get the next action from the policy
      action, _ = model.predict(obs, deterministic=True)
      # take an action based on the current observation
      obs, reward, done, info, _ = env.step(action)
      ep_rewards.append(reward)
  all_rewards.append(np.sum(ep_rewards))
print(f"Average reward: {np.mean(all_rewards)} over 5 episodes")
all_rewards

# Python testing

In [ ]:
import mujoco

In [ ]:
model = mujoco.MjModel.from_xml_path("../assets/wheelchair/myowc+arm.xml")
data = mujoco.MjData(model)

In [ ]:
model.ngeom

In [ ]:
model.geom_rgba

In [ ]:
[model.geom(i).name for i in range(model.ngeom)]

## `mjData`
`mjData` contains the *state* and quantities that depend on it. The state is made up of time, [generalized](https://en.wikipedia.org/wiki/Generalized_coordinates) positions and generalized velocities. These are respectively `data.time`, `data.qpos` and `data.qvel`. In order to make a new `mjData`, all we need is our `mjModel`

In [ ]:
[model.site(i).name for i in range(model.nsite)]

In [ ]:
print(data.site_xpos) #XYZ positions of sites listed above

In [ ]:
# JOINTS
print([model.jnt(i).name for i in range(model.njnt)])
print(model.njnt)

#joint position
print(data.qpos)
print(len(data.qpos))

#joint velocity
print(data.qvel)
print(len(data.qvel))

In [ ]:
# number of actuators, we don't have any because no muscles
model.na 

In [ ]:
data.act 
#action vector (muscle activations or actuator commands).